<a href="https://colab.research.google.com/github/munkefrugt/StockPrograms/blob/main/many_stocks_one_timeline_24_Marts.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
 !pip install yfinance;

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
import pandas as pd
import yfinance as yf
import numpy as np

import plotly.graph_objects as go

from plotly.subplots import make_subplots
import math


In [11]:
df_master = pd.DataFrame(columns=["Ticker", "Df", "Df_buy_signals","Num_of_signals", "Trades"])
tickers = ["AAPL","MSFT", "AMZN" ]#"MSFT", "AMZN"


def get_indicators(df):
    df.insert(0, "Row_Number", range(1, len(df) + 1))
    df = df.drop(df.columns[-1], axis=1)
    #df
    # Calculate technical indicators
    df['EMA50'] = df['Close'].ewm(span=50, adjust=False).mean()
    df['EMA100'] = df['Close'].ewm(span=100, adjust=False).mean()
    df['EMA200'] = df['Close'].ewm(span=200, adjust=False).mean()
    df['EMA500'] = df['Close'].ewm(span=500, adjust=False).mean()
    df['EMA2000'] = df['Close'].ewm(span=2000, adjust=False).mean()
    
    # Calculate MACD
    df['EMA12'] = df['Close'].ewm(span=12, adjust=False).mean()
    df['EMA26'] = df['Close'].ewm(span=26, adjust=False).mean()
    df['MACD'] = df['EMA12'] - df['EMA26']
    df['Signal'] = df['MACD'].ewm(span=9, adjust=False).mean()
    
    # Calculate Donchian channel
    window = 20
    df['Upper_band'] = df['High'].rolling(window=window).max()
    df['Stop_loss'] = df['Low'].rolling(window=window).min()

    # drop first stoploss nan rows
    df = df.iloc[window:]
    
    return df

def get_df_buy_signals(df):
    #buy signals: 
    window = 20
    df['Long_Entry'] = np.where(
      #locate latest max
      (df['MACD'] < 0) & 
      (df['Signal'] < 0) & 
      (df['MACD'].shift(1) < df['Signal'].shift(1)) & 
      (df['MACD'] > df['Signal']) &
      #close above ema200
      
      (df['Close'] > df['EMA200']) &
      (df['Row_Number'] > window),
      1, 0) 

    df_buy_signals = df[df['Long_Entry'] == 1]
    df_buy_signals = df_buy_signals.drop(
        ['Row_Number','Adj Close', 'EMA50','EMA100','EMA200','EMA500','EMA2000',
         'EMA12',	'EMA26' ,	'MACD', 	'Signal', 	'Upper_band','Long_Entry'],
          axis=1)

    return df, df_buy_signals

for ticker in tickers:
    # Download data
    df = yf.download(ticker, period="59d", interval='30m')

    df = get_indicators(df)
    df, df_buy_signals = get_df_buy_signals(df)
    num_long_entries = len(df_buy_signals)
    df_trades = pd.DataFrame(columns=["Trade_id","Status", "Entry_time", "Entry_price", "Entry_time", "Exit_price", "Type", "Cash","Equity"])
    df_trades.set_index("Trade_id", inplace=True)

    trades = []
    # Create a new row with the ticker, DataFrame, and start date
    new_row = {"Ticker": ticker, "Df": df, "Df_buy_signals": df_buy_signals, 
               "Num_of_signals": num_long_entries, "Df_trades": df_trades,"Trades": trades }
    
    # Add the new row to the DataFrame
    df_master = df_master.append(new_row, ignore_index=True)


df_master
#df_master.iloc[0]["Df"]



[*********************100%***********************]  1 of 1 completed


<ipython-input-11-fc2f3f3fdb3c>:71: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.



[*********************100%***********************]  1 of 1 completed


<ipython-input-11-fc2f3f3fdb3c>:71: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.



[*********************100%***********************]  1 of 1 completed


<ipython-input-11-fc2f3f3fdb3c>:71: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.



,Ticker,Df,Df_buy_signals,Num_of_signals,Trades,Df_trades
0,AAPL,Row_Number Open ...,Open High ...,9,[],"Empty DataFrame Columns: [Status, Entry_time, ..."
1,MSFT,Row_Number Open ...,Open High ...,6,[],"Empty DataFrame Columns: [Status, Entry_time, ..."
2,AMZN,Row_Number Open ...,Open High ...,7,[],"Empty DataFrame Columns: [Status, Entry_time, ..."


In [9]:
# df_trades = df_master.iloc[0]["Df_trades"]
# df_trades
# df_master.iloc[0]["Df"]
#df_buySignals = df_master.iloc[0]["Df_buy_signals"]
#df_buySignals['Status'] = 'open'
#df_buySignals['id'] = 'open'
#len(df_trades)

#df_buySignals

In [21]:
import math
timeline = df_master.iloc[0]["Df"].index

# keep all the detail sabout the trades
master_trades_df = pd.DataFrame(columns=[
    "trade_id", "cash", "cash_after_sell", "equity","cash_spend", "profit", "max_loss", "entry_time","entry_price", "exit_time","exit_price", 
    "position_size", "tp", "sl", "exit_type","ticker","status"])
#master_trades_df.index.name = "trade_id"

#is it used?
current_ticker_prices = pd.DataFrame(columns=['tickers', 'price', 'position_size','value'])

# Add tickers to the DataFrame
current_ticker_prices['tickers'] = tickers
current_ticker_prices['position_size'] = 0
current_ticker_prices['price'] = 0
current_ticker_prices['value'] = 0

cash_equity_df = pd.DataFrame(columns=['time', 'cash', 'equity'])
cash_equity_df['time'] = timeline
cash_equity_df['cash'] = 0
cash_equity_df['equity'] = 0
cash_equity_df.set_index('time', inplace=True)
 
start_capital = 10000
equity=None
cash = start_capital
risk_percentage = 0.005
# for trade objects
trades = []
winning_trades = 0
losing_trades = 0
current_winning_streak = 0
current_losing_streak = 0
max_winning_streak = 0
max_losing_streak = 0


class Trade:
    # initialize the class with the next ID
    next_id = 1

    def __init__(self, entry_price):
        self.entry_price = entry_price
        self.sl = None
        self.tp = None
        self.position_size = None
        self.entry_time = None
        self.current_open_price = None
        self.ticker = None
        self.trade_id = Trade.next_id  # assign the next available ID
        Trade.next_id += 1  # increment the next available ID
        self.status = "open"
         

for time in timeline:
    #store cash  equity for printing
    cash_equity_df.loc[time, "cash"] = cash
    cash_equity_df.loc[time, "equity"] = equity

    # Loop through the master row by row. check for signals.
    for index, row in df_master.iterrows():
        #print("index " + str(index))
        ticker = row["Ticker"]
        df = row["Df"]
        #df_buy_signals = row["Df_buy_signals"]
        #df_trades = row["Df_trades"]

        #trades = row["Trades"]
        
        # get the row of the df with the time we are at
        current_row = df.loc[df.index == time]
        signal = current_row["Long_Entry"].item() 
        open_price = current_row["Open"].item()
        current_price = open_price
        low_price = current_row["Low"].item() 
        high_price = current_row["High"].item() 
        sl = None
        
        # make an aproximate equity. 
        
        value = 0
        for trade in trades: 
          if trade.status== "open":
            value = value + trade.entry_price* trade.position_size
        aproximate_equity = value + cash
        equity = aproximate_equity

        # Check for buy signals
        if signal == 1:

          print("\n")
          print("Buy signal for ticker", ticker, "at time", time)
          print("equity: " + str(equity))

          #print(current_row)
          #print(total_position_amount)
          sl = current_row["Stop_loss"].item()
          print(cash)
          risk = open_price - sl

          if cash >= current_price and risk > 0: 
            max_position_size = (equity * risk_percentage) / risk
            position_size = math.floor(max_position_size)
            print("max position size "  + str(position_size))
            potential_spending = position_size * current_price

            max_loss = equity * risk_percentage
            potential_loss = position_size* risk

            if potential_spending > cash:
              # error its not the right position size. It need to be smaller. 
              position_size = math.floor(cash / current_price)
              potential_spending = position_size *current_price
              print("change position amount to " + str(position_size))
            else: 
              
              print("use max position size "  + str(position_size))

            #make the trade:
            if potential_loss <= max_loss :
              print("we can trade")
              print("potential_loss " + str(potential_loss))
              potential_loss_over_equity = potential_loss/ equity
              #df.loc[i, "Potential_loss_over_equity_pct"] = potential_loss_over_equity * 100

              print("potential_spending " + str(potential_spending))
              print("buy position size "  + str(position_size))

              cash =cash- potential_spending
              print("cash left " +str(cash))

              

              # Open a new trade
              print("Open Trade ! <-------------")

              # add trade
              trade = Trade(entry_price=open_price)
              trade.sl = sl
              trade.tp = open_price + risk*1.5
              trade.position_size = position_size
              trade.entry_time = time
              trade.ticker = ticker
              trades.append(trade) 
              print("trade_id : " + str(trade.trade_id))

              # fill master_trades_df
              trade_dict = {
                  "trade_id": trade.trade_id,
                  "cash" : cash,
                  "equity": equity,
                  "cash_spend": potential_spending,

                  #"profit": 0,
                  "max_loss": max_loss,
                  "entry_time": trade.entry_time,
                  "entry_price": trade.entry_price,
                  # "exit_time": ,
                  # "exit_price": ,
                  "position_size": trade.position_size,
                  "position*price": 0,
                  "cash after sell": 0,
                  "tp": trade.tp,
                  "sl": trade.sl,
                  #"type": "tp",
                  "ticker": ticker,
                  "status": trade.status

              }
              master_trades_df.loc[trade.trade_id] = trade_dict



              # df_trades["Trade_id"] = trade.trade_id
              #print(trade.trade_id)
              # df_trades[ "Time"] = time
              # df_trades[ "Buy_price"] = open_price
              # df_trades[ "Status"] = "open"
          else: 
            print(" ''''''''''''''''''''''''''''''''''''' not enough cash to trade or no risk availible, risk = : " + str(risk))
      


        for trade in trades:
            if trade.status == "open":
              #stoploss
              if low_price <= trade.sl:
                  print(f"Stopped out trade with entry price {trade.entry_price}")
                  print("trade_id : "+ str(trade.trade_id))
                  trade.status = "closed"
                  master_trades_df.loc[trade.trade_id,'status'] = "closed"
                  master_trades_df.loc[trade.trade_id,'exit_type'] = "sl"
                  master_trades_df.loc[trade.trade_id,'exit_time'] = time
                  # loss might be lower!
                  master_trades_df.loc[trade.trade_id,'exit_price'] = trade.sl
                  

                  #cash = cash + low_price * trade.position_size
                  master_trades_df.loc[trade.trade_id,'cash_after_sell'] = cash
                  cash_spend = trade.entry_price * trade.position_size 
                  #conservative
                  cash_return = trade.sl * trade.position_size 
                  profit = cash_return-cash_spend
                  master_trades_df.loc[trade.trade_id,'profit'] =  profit

                  # update cash: 
                  print("cash before update: " + str(cash))
                  cash = cash + cash_return
                  print("cash after update: " + str(cash))

                  print("profit: " + str(profit))
                  print("cash_return: " + str(cash_return) + " cash_spend: " + str(cash_spend))
                  print("entry price: " + str(trade.entry_price))
                  print("entry price: " + str(trade.entry_price))
                  

              #take profit
              elif high_price >= trade.tp:
                  print(f"Take profit hit for trade with entry price {trade.entry_price}")
                  print("trade_id : "+ str(trade.trade_id))

                  trade.status = "closed"
                  master_trades_df.loc[trade.trade_id,'status'] = "closed"
                  master_trades_df.loc[trade.trade_id,'exit_type'] = "tp"
                  master_trades_df.loc[trade.trade_id,'exit_time'] = time
                  master_trades_df.loc[trade.trade_id,'exit_price'] = trade.tp

                  cash = cash + trade.tp * trade.position_size
                  master_trades_df.loc[trade.trade_id,'cash_after_sell'] = cash
                  cash_spend = trade.entry_price * trade.position_size 
                  cash_return = trade.tp * trade.position_size 
                  master_trades_df.loc[trade.trade_id,'profit'] = cash_return-cash_spend 
 
print(current_ticker_prices)

#df_trades = df_master.iloc[0]["Df_trades"]
#df_trades
master_trades_df
#current_ticker_prices



Buy signal for ticker AAPL at time 2022-12-30 15:30:00
equity: 10000
10000
max position size 33
use max position size 33
we can trade
potential_loss 49.16992950439453
potential_spending 4254.359939575195
buy position size 33
cash left 5745.640060424805
Open Trade ! <-------------
trade_id : 1
Take profit hit for trade with entry price 128.9199981689453
trade_id : 1


Buy signal for ticker AMZN at time 2022-12-30 15:30:00
equity: 10073.754894256592
10073.754894256592
max position size 104
use max position size 104
we can trade
potential_loss 49.9195556640625
potential_spending 8626.799682617188
buy position size 104
cash left 1446.9552116394043
Open Trade ! <-------------
trade_id : 2
Take profit hit for trade with entry price 82.94999694824219
trade_id : 2


Buy signal for ticker MSFT at time 2023-01-03 14:30:00
equity: 10148.634227752686
10148.634227752686
max position size 32
use max position size 32
we can trade
potential_loss 50.240234375
potential_spending 7635.2001953125
buy po

,trade_id,cash,cash_after_sell,equity,cash_spend,profit,max_loss,entry_time,entry_price,exit_time,exit_price,position_size,tp,sl,exit_type,ticker,status
1,1,5745.640060,10073.754894,10000.000000,4254.359940,73.754894,50.000000,2022-12-30 15:30:00,128.919998,2022-12-30 15:30:00,131.154995,33,131.154995,127.430000,tp,AAPL,closed
2,2,1446.955212,10148.634228,10073.754894,8626.799683,74.879333,50.368774,2022-12-30 15:30:00,82.949997,2022-12-30 15:30:00,83.669991,104,83.669991,82.470001,tp,AMZN,closed
3,3,2513.434032,2513.434032,10148.634228,7635.200195,-50.240234,50.743171,2023-01-03 14:30:00,238.600006,2023-01-03 14:30:00,237.029999,32,240.955017,237.029999,sl,MSFT,closed
4,4,8501.254040,10173.348915,10098.393993,1597.139954,74.954922,50.491970,2023-01-06 11:00:00,84.059998,2023-01-06 11:30:00,88.004993,19,88.004993,81.430000,tp,AMZN,closed
5,5,6912.228798,10249.308937,10173.348915,3261.120117,75.960022,50.866745,2023-01-19 14:30:00,135.880005,2023-01-19 14:30:00,139.045006,24,139.045006,133.770004,tp,AAPL,closed
6,6,6484.109047,10325.508736,10249.308937,3765.199890,76.199799,51.246545,2023-01-19 15:00:00,94.129997,2023-01-19 15:30:00,96.034992,40,96.034992,92.860001,tp,AMZN,closed
7,7,4789.688560,10402.383736,10325.508736,5535.820175,76.875000,51.627544,2023-01-20 11:00:00,135.020004,2023-01-20 11:00:00,136.895004,41,136.895004,133.770004,tp,AAPL,closed
8,8,9197.233742,8141.673752,10402.383736,1205.149994,-50.650024,52.011919,2023-01-25 13:30:00,241.029999,2023-01-25 13:30:00,230.899994,5,256.225006,230.899994,sl,MSFT,closed
9,9,8141.673752,10424.993752,10402.383736,1055.559990,73.260040,52.011919,2023-01-25 13:30:00,95.959999,2023-01-25 14:00:00,102.620003,11,102.620003,91.519997,tp,AMZN,closed
10,10,6256.083824,6256.083824,10424.993752,4168.909927,-51.509979,52.124969,2023-01-31 10:30:00,245.229996,2023-01-31 10:30:00,242.199997,17,249.774994,242.199997,sl,MSFT,closed


#Plot

In [5]:
#df_master.iloc[0]["Df"]
cash_equity_df.tail()


,cash,equity
time,,
2023-03-23 13:30:00,1650.572121,1650.572121
2023-03-23 14:00:00,1650.572121,1650.572121
2023-03-23 14:30:00,1650.572121,1650.572121
2023-03-23 15:00:00,1650.572121,1650.572121
2023-03-23 15:30:00,1650.572121,1650.572121


In [ ]:
# make a dataframe for time cahs and equity
cash_equity_df

,cash,equity
time,,
2022-12-21 13:00:00,10000.00000,10000.00000
2022-12-21 13:30:00,10000.00000,10000.00000
2022-12-21 14:00:00,10000.00000,10000.00000
2022-12-21 14:30:00,10000.00000,10000.00000
2022-12-21 15:00:00,10000.00000,10000.00000
...,...,...
2023-03-16 13:30:00,6786.18671,6786.18671
2023-03-16 14:00:00,6786.18671,6786.18671
2023-03-16 14:30:00,6786.18671,6786.18671


In [22]:
df = df_master.iloc[0]["Df"]

df_trades = master_trades_df
fig = make_subplots(rows=6, cols=1, vertical_spacing=0.065, shared_xaxes=True, row_heights=[0.4, 0.1, 0.1, 0.1, 0.1, 0.1])



#donchian channel
fig.add_trace(go.Scatter(x=list(df.index), y=list(df.Stop_loss),line=dict(color='red', width=2), name='lower band'), 1, 1)

#add buy signals: 

#sell stoploss reached
jitter = 0.25  
y = df_trades['exit_price']
y_jitter = y + np.random.normal(0, jitter, size=len(y))

fig.add_trace(
    go.Scatter(x=df_trades['exit_time'], y=y_jitter, mode='markers',
                  marker=dict(size=30, color='red', symbol='triangle-down')),1,1)

# buy signal

fig.add_trace(
    go.Scatter(x=df_trades['entry_time'], y=df_trades['entry_price'], mode='markers',
                  marker=dict(size=30, color='green', symbol='triangle-up')),1,1)



fig.add_trace(go.Scatter(x=list(df.index), y=list(df.Close),line=dict(color='blue', width=2), name='Close'), 1, 1)


fig.add_trace(go.Scatter(x=list(df.index), y=list(df.EMA200),line=dict(color='red', width=2), name='EMA200' ), 1, 1)
fig.add_trace(go.Scatter(x=list(df.index), y=list(df.EMA500),line=dict(color='green', width=2), name='EMA500' ), 1, 1)

fig.add_trace(go.Scatter(x=list(df.index), y=list(df.EMA2000),line=dict(color='brown', width=2), name='EMA2000' ), 1, 1)

#second plot
fig.add_trace(go.Scatter(x=list(df.index), y=list(df.MACD),line=dict(color='orange', width=2), name='MACD'), 2, 1);
fig.add_trace(go.Scatter(x=list(df.index), y=list(df.Signal),line=dict(color='blue', width=2), name='Signal'), 2, 1);

# cash
fig.add_trace(go.Scatter(x=list(cash_equity_df.index), y=list(cash_equity_df.cash),line=dict(color='blue', width=2), name='Cash'), 3, 1);

fig.update_yaxes(
    title_text="Cash",
    title_font=dict(size=25, family="Arial", color="blue"),
    row=3, col=1
)

# equity
fig.add_trace(go.Scatter(x=list(cash_equity_df.index), y=list(cash_equity_df.equity),line=dict(color='blue', width=2), name='Equity'), 4, 1);

fig.update_yaxes(
    title_text="Equity",
    title_font=dict(size=25, family="Arial", color="blue"),
    row=4, col=1
)

# # positions
# fig.add_trace(go.Scatter(x=list(cash_equity_df.time), y=list(cash_equity_df.position_amount),line=dict(color='blue', width=2), name='Total_position_amount'), 5, 1);

# fig.update_yaxes(
#     title_text="Total_position",
#     title_font=dict(size=25, family="Arial", color="blue"),
#     row=5, col=1
# )


# # potential loss

# fig.add_trace(go.Scatter(x=list(df.index), y=list(df.Potential_loss_over_equity_pct),line=dict(color='blue', width=2), name='Total_position_amount'), 6, 1);

# fig.update_yaxes(
#     title_text="potential loss %",
#     title_font=dict(size=25, family="Arial", color="blue"),
#     row=6, col=1
# )

fig.update_layout(height=1000, width=1200, showlegend=True)


fig.update_layout({
    'plot_bgcolor': 'rgba(0,0,0,0)',
    'paper_bgcolor': 'rgba(0,0,0,0)'
})

fig.update_layout(
    legend=dict(
        font=dict(
            family="sans-serif",
            size=18,
            color="blue"
        )
    )
)

fig.update_layout(
    xaxis=dict(
        type="date"),
    #xaxis2_rangeslider_visible=True,
    xaxis2_type="date"
    );
fig.update_xaxes(
    rangebreaks=[
        dict(bounds=["sat", "mon"]),
        dict(bounds=[16, 9.50], pattern="hour"), #hide hours outside of 9am-5pm
    ]
)
fig.update_xaxes(tickfont=dict(size=20, family='Arial', color='blue'))

# Update the y-axis tick label font size and style
fig.update_yaxes(tickfont=dict(size=20, family='Arial', color='blue'))

fig.show()



In [7]:
# Initialize an empty figure
fig = go.Figure()

# Loop through each ticker and plot its data
for ticker in tickers:
    df = df_master[df_master["Ticker"] == ticker]["Df"].iloc[0]
    df_trades = master_trades_df[master_trades_df["ticker"] == ticker]

    fig.add_trace(go.Scatter(x=list(df.index), y=list(df.Close), name=f"{ticker} Close"))

    # Add buy signals
    fig.add_trace(go.Scatter(x=df_trades['entry_time'], y=df_trades['entry_price'], mode='markers',
                             marker=dict(size=30, color='green', symbol='triangle-up'), name=f"{ticker} Buy Signal"))

    # Add sell signals when stoploss is reached
    jitter = 0.25
    y = df_trades['exit_price']
    y_jitter = y + np.random.normal(0, jitter, size=len(y))

    fig.add_trace(go.Scatter(x=df_trades['exit_time'], y=y_jitter, mode='markers',
                             marker=dict(size=30, color='red', symbol='triangle-down'), name=f"{ticker} Sell Signal"))

# Update the layout and show the figure
fig.update_layout(title="Stock Prices and Buy/Sell Signals", xaxis_title="Date", yaxis_title="Price")
fig.show()

#extra advanced equity


In [ ]:
#calculate equity for advanced real: 
#insert this in the time loop
#get equity:

    for index, row in df_master.iterrows():
        ticker = row["Ticker"]
        df = row["Df"]
        current_row = df.loc[df.index == time]
        open_price = current_row["Open"].item()

        for trade in trades:
          if trade.status == "open" and trade.ticker == ticker:
              
              #if ticker in current_ticker_prices.index:
              current_ticker_prices.loc[current_ticker_prices['tickers'] == ticker, 'price'] = open_price
              current_ticker_prices.loc[current_ticker_prices['tickers'] == ticker, 'position_size'] = trade.position_size
              current_ticker_prices.loc[current_ticker_prices['tickers'] == ticker, 'value'] = trade.position_size * open_price

          #else:
                  #print(f"No ticker {ticker} found in current_ticker_prices")
        sum =  current_ticker_prices['value'].sum()
        #print(sum)
        equity = cash + sum
        
        #print(equity)
        #print(current_ticker_prices)

In [ ]:
import pandas as pd

# Create a list of tickers
tickers = ['AAPL', 'GOOG']

# Create an empty DataFrame
df = pd.DataFrame(columns=['tickers', 'price', 'position_size','value'])

# Add tickers to the DataFrame
df['tickers'] = tickers

# # Set initial price and position size for each ticker
# df['price'] = [0.0] * len(tickers)
# df['position_size'] = [0] * len(tickers)

df.loc[df['tickers'] == 'AAPL', 'price'] = 150.0
df.loc[df['tickers'] == 'AAPL', 'position_size'] = 5
df.loc[df['tickers'] == 'AAPL', 'value'] = 5*150

df.loc[df['tickers'] == 'GOOG', 'price'] = 70.0
df.loc[df['tickers'] == 'GOOG', 'position_size'] = 3
df.loc[df['tickers'] == 'GOOG', 'value'] = 3*70

print(df)
df['value'].sum()

  tickers  price position_size value
0    AAPL  150.0             5   750
1    GOOG   70.0             3   210


960

In [ ]:
portfolio

,ticker,price_open,positions,time,equity
0,AMZN,91.010002,NaN,2023-03-10 15:30:00,NaN


In [ ]:
df_trades = df_master.iloc[3]["Df_trades"]
df_trades


IndexError: ignored

In [ ]:
df_master.iloc[0]["Df_trades"].index
#df_master

Index([], dtype='object', name='Trade_id')

#Extra

In [ ]:
#df_master.iloc[0]["Df"]["Long_Entry"]
#df_master.iloc[0]["Df"].index
#df_master.iloc[0]["Df"]

In [ ]:
df_master
#df_master.iloc[0]["Trades"].append(1) 
#df_master.iloc[0]["Trades"]
 

#Experiments

In [ ]:
#df_master
#df_master.iloc[0]["Trades"].values[0] 
df_master.loc[df_master["Ticker"] == "AAPL", "Trades"].iloc[0][0].sl
lenght = len(df_master.loc[df_master["Ticker"] == "AAPL", "Trades"].iloc[0])
#df_master.loc[df_master["Ticker"] == "AAPL", "Trades"].iloc[0][0].status
# for i in range(len(df_master.loc[df_master["Ticker"] == "AAPL", "Trades"].iloc[0])) 
#   df_master.loc[df_master["Ticker"] == "AAPL", "Trades"].iloc[0][i].status
for i in range(lenght):
  print(df_master.loc[df_master["Ticker"] == "AAPL", "Trades"].iloc[0][i].status)


closed
closed
closed
closed
closed
closed
closed
closed
closed
closed
closed
closed
closed
closed


In [ ]:
df_master.iloc[1]["Df_buy_signals"]

,Row_Number,Open,High,Low,Close,Adj Close,EMA50,EMA100,EMA200,EMA500,EMA2000,EMA12,EMA26,MACD,Signal,Upper_band,Stop_Loss,Long_Entry
Datetime,,,,,,,,,,,,,,,,,,
2023-01-25 13:30:00,438,241.029999,241.500000,239.960007,240.330002,240.330002,239.647441,238.402459,237.466066,239.722703,245.023318,239.358256,239.869346,-0.511090,-0.610404,243.470001,230.899994,1
2023-01-31 10:30:00,484,245.229996,245.554993,244.539993,244.710007,244.710007,244.062992,242.385332,240.263674,240.619311,245.024774,243.951095,244.407341,-0.456245,-0.510671,249.830002,242.199997,1
2023-02-07 09:30:00,547,260.529999,263.450012,260.079987,260.799988,260.799988,256.237542,252.476810,247.551953,243.917611,245.645778,257.594800,257.668914,-0.074115,-0.172648,263.450012,254.779999,1
2023-02-10 15:30:00,598,261.739990,263.529999,261.605011,263.100006,263.100006,263.728225,260.746976,254.786465,247.954530,246.650569,262.521909,263.714940,-1.193031,-1.205542,268.829987,260.661987,1


In [ ]:
ticker = "MSFT"
DataframeAPPL = df_master.loc[df_master["Ticker"] == ticker, "Df"].iloc[0]
#ticker = "AMZN"
buysignals = df_master.loc[df_master["Ticker"] == ticker, "Df_buy_signals"].iloc[0]
buysignals


,Row_Number,Open,High,Low,Close,Adj Close,EMA50,EMA100,EMA200,EMA500,EMA2000,EMA12,EMA26,MACD,Signal,Upper_band,Stop_Loss,Long_Entry
Datetime,,,,,,,,,,,,,,,,,,
2023-01-25 13:30:00,464,241.029999,241.500000,239.960007,240.330002,240.330002,239.647442,238.402904,237.500062,240.174239,246.659236,239.358256,239.869346,-0.511090,-0.610404,243.470001,230.899994,1
2023-01-31 10:30:00,510,245.229996,245.554993,244.539993,244.710007,244.710007,244.062992,242.385509,240.285135,240.994960,246.587144,243.951095,244.407341,-0.456245,-0.510671,249.830002,242.199997,1
2023-02-07 09:30:00,573,260.529999,263.450012,260.079987,260.799988,260.799988,256.237542,252.476860,247.563383,244.209582,247.112756,257.594800,257.668914,-0.074115,-0.172648,263.450012,254.779999,1
2023-02-10 15:30:00,624,261.739990,263.529999,261.605011,263.100006,263.100006,263.728225,260.746994,254.793328,248.192621,248.044606,262.521909,263.714940,-1.193031,-1.205542,268.829987,260.661987,1
